In [10]:
import numpy as np
import signal
import time
import pickle
import sys
import matplotlib.pyplot as plt
import math

In [12]:
def foo(x,
    G = np.array([
        [16, 0, 0, -6, -2, 1],
        [0, 12, -2, 1, 1, 4],
        [0, -2, 14, 1, -3, -6],
        [-6, 1, 1, 8, 2, -4],
        [-2, 1, -3, 2, 6, 0],
        [1, 4, -6, -4, 0, 6]
    ]),
    C = np.array([[10, -30, 20, -20, -20, 10]]).T):

    return (x.T @ G @ x) / 2 + C.T @ x

def fooGradient(x, G, C):
    return G @ x + C

#### Lagrangian Dual Problem

In [15]:
def qoo(v, G, A ,C, b):
    return (-(A.T @ v - C).T @ np.linalg.inv(G) @ (A.T @ v - C)) / 2 + (b.T @ v)

def qooGradient(v, G, A, C, b):
    return -A @ np.linalg.inv(G) @ (A.T @ v - C) + b

def armijo(x, G, A, C, b, alpha, direction):
    f1 = qoo(x, G, A, C, b)
    f2 = qoo(x + alpha * direction, G, A, C, b)
    # print("x=", x, "f1=", f1, "f2=", f2, "alpha=", alpha, "direction=", direction)
    return f2 > f1

def projection(v):
    return np.clip(v, a_min=0, a_max=None)

def happy(x, next_x):
    criteria = (np.abs(x - next_x) < 1e-8)
    # print(criteria)
    return np.all(criteria)

def Dual():
    x = np.zeros([12,1])
    G = np.array([
        [16, 0, 0, -6, -2, 1],
        [0, 12, -2, 1, 1, 4],
        [0, -2, 14, 1, -3, -6],
        [-6, 1, 1, 8, 2, -4],
        [-2, 1, -3, 2, 6, 0],
        [1, 4, -6, -4, 0, 6]
    ])
    C = np.array([[10, -30, 20, -20, -20, 10]]).T
    A = np.array([[np.identity(6)], [-np.identity(6)]])
    A = A.reshape([12, 6])
    b = -np.ones([12, 1])
    rho = 0.5
    alpha_0 = 0.05
    k = 1

    while True:
        alpha = alpha_0
        gradX = qooGradient(x, G, A, C, b)
        direction = gradX
        while not armijo(x, G, A, C, b, alpha, direction):
            alpha *= rho
        if happy(x, projection(x + alpha * direction)):
            break
        # next step
        x = projection(x + alpha * direction)
        k = k + 1
        # print("k:", k, "x:", x, "alpha:", alpha, "gradX:", gradX, "direction:", direction)
    return x, np.linalg.inv(G) @ (A.T @ x - C)

v, x = Dual()
print("(x, v) = ", x, v)
print("min foo = ", foo(x))

(x, v) =  [[-0.06250001]
 [ 1.00000012]
 [-1.0000002 ]
 [ 0.99999997]
 [ 1.00000005]
 [-0.99999987]] [[ 0.        ]
 [ 0.        ]
 [ 7.99999601]
 [ 0.        ]
 [ 0.        ]
 [ 9.93750256]
 [ 0.        ]
 [17.99999767]
 [ 0.        ]
 [ 5.62500069]
 [ 7.87499903]
 [ 0.        ]]
min foo =  [[-75.03125262]]


#### Primal problem
we can get the x and min foo are the same

In [9]:
def armijo(x, alpha, direction, G, C):
    f1 = foo(x, G, C)
    f2 = foo(x + alpha * direction, G, C)
    # print("x=", x, "f1=", f1, "f2=", f2, "alpha=", alpha, "direction=", direction)
    return f2 <= f1

def happy(x, next_x):
    criteria = (np.abs(x - next_x) < 1e-8)
    return np.all(criteria)

def checkKKT(x, G, C, u=1, l=-1):
    Gx_plus_C = G @ x + C
    ll = np.maximum(Gx_plus_C, 0)  # 將所有元素小於0的部分補0
    lu = np.maximum(-Gx_plus_C, 0)  # 將所有元素大於等於0的部分補0
    print("Original:", Gx_plus_C)
    print("ll:", ll)
    print("lu:", lu)
    print("x", x)
    print("lu*(u-x)", lu.T@(u-x))
    print("ll*(x-l)", ll.T@(x-l))

def projection(x, u=1, l=-1):
    return np.clip(x, l, u)

def main():
    rho = 0.5
    alpha_0 = 0.05
    k = 1
    x = np.array([
    [0],
    [0],
    [0],
    [0],
    [0],
    [0]
])
    G = np.array([
        [16, 0, 0, -6, -2, 1],
        [0, 12, -2, 1, 1, 4],
        [0, -2, 14, 1, -3, -6],
        [-6, 1, 1, 8, 2, -4],
        [-2, 1, -3, 2, 6, 0],
        [1, 4, -6, -4, 0, 6]
    ])
    C = np.array([[10, -30, 20, -20, -20, 10]]).T
    while True:
        alpha = alpha_0
        gradX = fooGradient(x, G, C)
        direction = -gradX
        while not armijo(x, alpha, direction, G, C):
            alpha *= rho
        if happy(x, projection(x + alpha * direction)):
            break
        # next step
        x = projection(x + alpha * direction)
        k = k + 1
        # print(k, x, p, q, h, fooHessian(x), alpha, gradX, direction)

    # Check KKT
    checkKKT(x, G, C)
    print()
    print("k=", k)
    print(foo(x, G, C))
    return 0
temp = main()

Original: [[-1.84320001e-07]
 [-1.80000000e+01]
 [ 8.00000000e+00]
 [-5.62499993e+00]
 [-7.87499998e+00]
 [ 9.93749999e+00]]
ll: [[0.        ]
 [0.        ]
 [8.        ]
 [0.        ]
 [0.        ]
 [9.93749999]]
lu: [[1.84320001e-07]
 [1.80000000e+01]
 [0.00000000e+00]
 [5.62499993e+00]
 [7.87499998e+00]
 [0.00000000e+00]]
x [[-0.06250001]
 [ 1.        ]
 [-1.        ]
 [ 1.        ]
 [ 1.        ]
 [-1.        ]]
lu*(u-x) [[1.95840003e-07]]
ll*(x-l) [[0.]]

k= 13
[[-75.03125]]
